In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import tensorflow as tf
import os,glob
from PIL import Image

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split

np.random.seed(3)

In [2]:
def plot_loss_curve(history):
    plt.figure(figsize=(15, 10))

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()   
    
    print("train loss=", history.history['loss'][-1])
    print("validation loss=", history.history['val_loss'][-1])
    
def plot_acc_curve(history):
    plt.figure(figsize=(15, 10))

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()   
    
    print("train accuracy=", history.history['accuracy'][-1])
    print("validation accuracy=", history.history['val_accuracy'][-1])

In [21]:
X_train, X_test, y_train, y_test, X_val, y_val = np.load("C:/Users/csms4/jupyter/multi_image_data.npy",allow_pickle=True)

X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [22]:
model = Sequential()
model.add(Input(shape=(300,300,3), name='input_layer'))
model.add(preprocessing.Rescaling(1./255))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', name='conv_layer1')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(rate=0.5))

model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', name='conv_layer2')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(rate=0.5)) 

model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu', name='conv_layer3')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(rate=0.5)) 
model.add(Flatten())

model.add(Dense(256, activation='relu', name='hidden_layer1')) 
model.add(Dropout(rate=0.5)) 
model.add(Dense(128, activation='relu', name='hidden_layer2')) 
model.add(Dropout(rate=0.5)) 
model.add(Dense(3, activation='softmax', name='output_layer'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
early_stopping = EarlyStopping(patience = 5) # 조기종료 콜백함수 정의

history = model.fit(
        X_train, y_train, validation_data=(X_val, y_val), batch_size=100, epochs=20,
        callbacks=[early_stopping])

Epoch 1/20
144/144 [==============================] - 585s 4s/step - loss: 0.9835 - accuracy: 0.4810 - val_loss: 0.9476 - val_accuracy: 0.5517
Epoch 2/20
144/144 [==============================] - 584s 4s/step - loss: 0.8721 - accuracy: 0.5883 - val_loss: 0.9658 - val_accuracy: 0.5208
Epoch 3/20
144/144 [==============================] - 584s 4s/step - loss: 0.7949 - accuracy: 0.6480 - val_loss: 0.9202 - val_accuracy: 0.5881
Epoch 4/20
144/144 [==============================] - 583s 4s/step - loss: 0.7177 - accuracy: 0.7018 - val_loss: 0.9408 - val_accuracy: 0.5733
Epoch 5/20
 62/144 [===========>..................] - ETA: 5:15 - loss: 0.6121 - accuracy: 0.7589

KeyboardInterrupt: 

In [ ]:
plot_loss_curve(history)
plot_acc_curve(history)

model.save('cnn07')

In [22]:
model = load_model("cnn04")
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 300, 300, 3)       0         
_________________________________________________________________
conv_layer1 (Conv2D)         (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 150, 150, 32)      0         
_________________________________________________________________
conv_layer2 (Conv2D)         (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 75, 75, 64)       

In [7]:
# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate(X_test, y_test)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
141/141 [==============================] - 39s 275ms/step - loss: 0.7752 - accuracy: 0.8033
accuracy: 80.33%
